Anggota Kelompok (Kelas N (Prodi RKA)):

- Chelsea (5054241010)
- Wayan Raditya Putra (5054241029)
- Syauqi Nabil Tasri (5054241040)

---

**Dependencies Installing**

Sebelum memulai projek, alangkah baiknya jika kita menyiapkan dan menginstall tools - tools yang diperlukan.

In [ ]:
!pip -q install requests beautifulsoup4 lxml dateparser

**lxml** adalah pustaka pengurai **XML** dan **HTML** berkinerja tinggi untuk Python, yang dikenal karena kecepatan dan rangkaian fiturnya yang lengkap.

Library ini menyediakan cara yang cepat dan efisien untuk mengurai, memanipulasi, dan mengekstrak data dari berkas **XML** dan **HTML** menggunakan **API** mirip ElementTree yang dipadukan dengan kecepatan pustaka libxml2 dan libxslt. lxml banyak digunakan dalam pengikisan web, ekstraksi data, dan tugas-tugas lain yang memerlukan penanganan data terstruktur dari sumber **XML** atau **HTML**.

**Import Library**

In [ ]:
import re, json, time, random
from typing import List, Dict, Any, Optional
from urllib.parse import urlparse, urljoin, urlencode, parse_qs

import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import urllib.robotparser as robotparser
import dateparser
from datetime import datetime, date
import difflib
from collections import defaultdict
from urllib.parse import urlparse, urlunparse
import pandas as pd

Penjelasan

1. re, json, time, random: Untuk ekspresi reguler, pengolahan JSON, pengaturan waktu, dan fungsi acak.
2. typing: Memberikan petunjuk tipe data agar kode lebih jelas.
3. urllib.parse: Untuk memparsing dan memanipulasi URL.
4. requests: Untuk mengirim permintaan HTTP.
5. BeautifulSoup: Untuk memparsing dan mengekstrak data dari HTML.
6. HTTPAdapter, Retry: Untuk mengatur pengulangan permintaan HTTP agar lebih handal.
7. robotparser: Untuk memeriksa aturan robots.txt pada situs web agar crawler mematuhi batasan.
8. dateparser, datetime: Untuk memparsing dan mengelola tanggal dan waktu.
9. difflib: Untuk membandingkan urutan data, berguna untuk pencocokan yang mirip.
10. defaultdict: Tipe dictionary yang memudahkan pengelolaan nilai default.




**Configuration Setting**

Disini kita perlu menyesuaikan beberapa konfigurasi yang dibutuhkan nanti sepert url, max page yang dibaca, timeout, veriabel untuk filtering (keyword, start date, end date), dan yang lainnya.

In [ ]:
SECTION_URL = "https://nasional.kompas.com/"

need_date_from_detail = True
RESPECT_ROBOTS = True
HARD_CAP       = 10
DELAY_S        = (0.7, 1.6)
TIMEOUT_S      = 20
UA             = "TestScraper/1.0"
LANG_HDR       = "id,en;q=0.9"

KEYWORDS = ["sahroni"]
START_DATE = "2025-01-09"
END_DATE   = "2025-04-09"

MATCH_IN = "title"
CASE_SENSITIVE = False
INCLUDE_WITHOUT_DATE = False

dalam configure setting ini kita perlu menyesuaikan beberapa konfigurasi yang dibutuhkan nanti sepert url, max page yang dibaca, timeout, veriabel untuk filtering (keyword, start date, end date), dan yang lainnya.

nah maka dari itu kode ini dipakai dalam mengatur parameter untuk scraper berita yang akan mengambil maks 10 artikel dari halaman nasional kompas pada tanggal 9 Jan 2025, mencari kata kunci tertentu di judul artikel, dengan pengaturan agar scraper dapat memahami aturan robots.txt, menggunakan delay acak antar permintaan, dan mengambil tanggal berita dari halaman detail jika diperlukan

**Session HTTP**

Dalam konteks session **HTTP**, fitur dari library requests di Python, di mana objek Session digunakan untuk mengelola dan mempertahankan koneksi **HTTP** secara efisien selama beberapa permintaan. Kode ini mengatur sesi **HTTP** yang handal dengan mekanisme retry dan header khusus menggunakan objek **requests.Session()** untuk menjaga koneksi tetap efisien dan konsisten.
Selain itu, kode ini juga memeriksa aturan **robots.txt** sebelum mengakses halaman, mengambil dan memparsing halaman **HTML** menjadi objek BeautifulSoup, serta mengonversi string tanggal dalam bahasa Indonesia atau Inggris menjadi objek tanggal Python yang dapat digunakan untuk filter atau analisis data.
Semua ini merupakan bagian penting dalam membuat scraper web yang efisien, sopan, dan akurat.

In [ ]:
session = requests.Session()
retries = Retry(total=4, backoff_factor=0.5,
                status_forcelist=[429, 500, 502, 503, 504],
                allowed_methods=["GET"])
session.mount("http://", HTTPAdapter(max_retries=retries))
session.mount("https://", HTTPAdapter(max_retries=retries))
session.headers.update({
    "User-Agent": UA,
    "Accept-Language": LANG_HDR,
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
})

def is_allowed_by_robots(url: str, user_agent: str = UA) -> bool:
    """Cek robots.txt; kalau gagal baca → anggap tidak boleh (konservatif)."""
    try:
        p = urlparse(url)
        robots_url = f"{p.scheme}://{p.netloc}/robots.txt"
        rp = robotparser.RobotFileParser()
        rp.set_url(robots_url); rp.read()
        return bool(rp.can_fetch(user_agent, url))
    except Exception:
        return False

def get_soup(url: str) -> BeautifulSoup:
    """HTTP GET + parse HTML ke BeautifulSoup (parser lxml)."""
    r = session.get(url, timeout=TIMEOUT_S)
    r.raise_for_status()
    return BeautifulSoup(r.text, "lxml")

def parse_id_date(s: Optional[str]):
    """Parse string tanggal (ID/EN) → datetime.date, atau None."""
    if not s: return None
    dt = dateparser.parse(
        s,
        languages=["id", "en"],
        settings={"DATE_ORDER": "DMY", "TIMEZONE": "Asia/Jakarta", "RETURN_AS_TIMEZONE_AWARE": False}
    )
    return dt.date() if dt else None

In [ ]:
#@title Extractor Function

def _jsonld_blocks(soup: BeautifulSoup) -> List[dict]:
    blocks = []
    for sc in soup.find_all("script", type="application/ld+json"):
        raw = sc.string or ""
        try:
            data = json.loads(raw)
        except Exception:
            continue
        if isinstance(data, list):
            blocks.extend([x for x in data if isinstance(x, dict)])
        elif isinstance(data, dict):
            blocks.append(data)
    return blocks

def _is_article_url(href: str, base_url: str) -> bool:
    """Validasi URL artikel (domain sama + pola umum)."""
    if not href: return False
    abs_url = urljoin(base_url, href)
    u = urlparse(abs_url)
    if u.netloc != urlparse(base_url).netloc: return False
    q = (u.query or "").lower()
    if "utm_" in q or "source=navbar" in q: return False
    # Pola umum Kompas/KompasTV untuk artikel:
    return bool(re.search(r"/read/|/berita/|/video/", u.path))

def extract_from_itemlist(soup: BeautifulSoup, base_url: str) -> List[Dict[str, Any]]:
    """Ambil artikel dari JSON-LD ItemList/Collection/SearchResults."""
    rows = []
    for b in _jsonld_blocks(soup):
        typ = b.get("@type")
        if isinstance(typ, list): typ = typ[0] if typ else None
        if str(typ).lower() in ("itemlist","collectionpage","searchresultspage"):
            for el in b.get("itemListElement", []):
                node = el.get("item", el)
                if not isinstance(node, dict): continue
                url = node.get("url") or node.get("@id")
                name = node.get("name") or node.get("headline") or node.get("title")
                dp   = node.get("datePublished") or node.get("dateCreated")
                if url and _is_article_url(url, base_url):
                    rows.append({
                        "title": (name or "").strip(),
                        "url": urljoin(base_url, url),
                        "published_raw": dp,
                        "summary": node.get("description"),
                    })
    return list({r["url"]: r for r in rows}.values())

def extract_from_dom(soup: BeautifulSoup, base_url: str) -> List[Dict[str, Any]]:
    """Fallback DOM: cari anchor artikel di area <main>."""
    rows = []
    main = soup.select_one("main") or soup
    for a in main.select("a[href]"):
        href = a.get("href")
        if not _is_article_url(href, base_url):
            continue
        title = a.get_text(" ", strip=True)
        if not title:
            continue
        url = urljoin(base_url, href)
        # Timestamp di sekitar kartu
        ts = None
        card = a.find_parent(["article","div","li"])
        if card:
            t = card.find("time")
            if t:
                ts = t.get("datetime") or t.get_text(" ", strip=True)
            else:
                # KompasTV kadang pakai span teks tanggal (mis. "1 September 2025")
                near = card.find(lambda tag: tag.name in ("span","div","p")
                                 and re.search(r"\b\d{1,2}\s\w+\s\d{4}\b|\bWIB\b", tag.get_text(" ", strip=True)))
                if near:
                    ts = near.get_text(" ", strip=True)
        rows.append({"title": title, "url": url, "published_raw": ts})
    return list({r["url"]: r for r in rows}.values())

def parse_article_detail(url: str) -> Dict[str, Any]:
    """Ambil judul/tanggal/body dari halaman artikel (selector → meta → JSON-LD → URL path)."""
    s = get_soup(url)

    # Bersihkan noise
    for junk in s.select("script, style, .ads, .advertisement, .share, .social-share"):
        junk.decompose()

    # Judul
    title_el = s.select_one("h1") or s.select_one("h1.article__title")
    title = title_el.get_text(" ", strip=True) if title_el else None

    # Tanggal (urutan prioritas)
    published_raw = None
    t = s.select_one("time")
    if t:
        published_raw = t.get("datetime") or t.get_text(" ", strip=True)

    if not published_raw:  # meta tag umum
        for css in [
            'meta[itemprop="datePublished"]',
            'meta[property="article:published_time"]',
            'meta[name="date"]',
            'meta[name="publishdate"]',
            'meta[name="pubdate"]',
            'meta[property="og:published_time"]',
            'meta[property="article:modified_time"]',
        ]:
            m = s.select_one(css)
            if m and m.has_attr("content") and m["content"]:
                published_raw = m["content"].strip()
                break

    if not published_raw or not title:  # JSON-LD NewsArticle
        for b in _jsonld_blocks(s):
            typ = b.get("@type")
            if isinstance(typ, list): typ = typ[0] if typ else None
            if str(typ).lower() in ("newsarticle","article","blogposting"):
                title = title or b.get("headline") or b.get("name") or b.get("title")
                published_raw = published_raw or b.get("datePublished") or b.get("dateCreated")
            if title and published_raw:
                break

    if not published_raw:  # fallback URL path /YYYY/MM/DD/
        m = re.search(r"/(\d{4})/(\d{2})/(\d{2})/", url)
        if m:
            published_raw = f"{m.group(1)}-{m.group(2)}-{m.group(3)}"

    # Body (opsional)
    body = None
    for sel in ["article .read__content", "article .article__body", ".article__content", "article"]:
        el = s.select_one(sel)
        if el:
            body = el.get_text(" ", strip=True)
            if body: break

    return {"title": title, "published_raw": published_raw, "text": body}

**Extractor Function (mengambil (mengekstrak) informasi artikel dari halaman web)**

1. `_jsonld_blocks(soup)`  
Mengambil semua blok **JSON-LD** dari halaman HTML tag `<script type="application/ld+json">`,  
lalu mengembalikan daftar objek JSON yang valid.  
**Tujuan:** ambil metadata artikel yang sering disimpan dalam format JSON-LD.  

2. `_is_article_url(href, base_url)`  
Validasi apakah sebuah URL benar-benar link artikel, dengan syarat:  
- Masih dalam domain yang sama  
- Bukan link tracking (`utm_`, `source=navbar`)  
- Path mengandung `/read/`, `/berita/`, atau `/video/`  
**Tujuan:** filter agar hanya dapat URL artikel asli.  

3. `extract_from_itemlist(soup, base_url)`  
Mengekstrak daftar artikel dari blok JSON-LD bertipe **ItemList**, **CollectionPage**, atau **SearchResultsPage**.  
Mengambil:
- Judul  
- URL  
- Tanggal publikasi  
- Ringkasan artikel  

4. `extract_from_dom(soup, base_url)`  
*fallback* jika JSON-LD tidak ada.  
Mencari link artikel langsung di dalam elemen `<main>` pada HTML:  
- Cek anchor `<a>` yang memenuhi kriteria URL artikel  
- Ambil teks link sebagai judul  
- Coba cari tanggal di tag `<time>` atau teks tanggal dalam `<span>/<div>`  
**Tujuan:** alternatif scraping langsung dari HTML DOM.  

5. `parse_article_detail(url)`  
Mengambil detail isi artikel dari halaman tertentu:  
- **Judul** → dari `<h1>` atau JSON-LD  
- **Tanggal** → dari `<time>`, meta tag umum, JSON-LD, atau fallback ke URL `/YYYY/MM/DD/`  
- **Isi teks** → dari `article .read__content`, `.article__body`, `.article__content`, atau `article`  
**Hasil:** `{title, published_raw, text}`  


In [ ]:
#@title Pagination Function

def _replace_page_param(url: str, page_num: int) -> str:
    """Set/ubah ?page=N pada URL (menjaga path & query lain)."""
    u = urlparse(url)
    qs = parse_qs(u.query)
    qs["page"] = [str(page_num)]
    new_q = urlencode({k: (v[0] if len(v)==1 else v) for k,v in qs.items()}, doseq=True)
    return u._replace(query=new_q).geturl()

def _extract_page_numbers_from_dom(soup: BeautifulSoup) -> List[int]:
    """Kumpulkan kandidat nomor halaman dari anchor pagination."""
    nums = set()
    for a in soup.find_all("a", href=True):
        t = (a.get_text(strip=True) or "")
        if t.isdigit():
            nums.add(int(t))
        m = re.search(r"[?&]page=(\d+)", a["href"])
        if m:
            nums.add(int(m.group(1)))
        if re.search(r"(Terakhir|Last|>>|»)", t, re.I):
            m2 = re.search(r"[?&]page=(\d+)", a["href"])
            if m2:
                nums.add(int(m2.group(1)))
    return sorted(n for n in nums if n > 0)

def _find_next_by_dom(soup: BeautifulSoup, current_url: str) -> Optional[str]:
    """Cari tautan halaman berikutnya lewat DOM (rel=next / teks Next/›/»/Selanjutnya/Berikutnya)."""
    a = soup.select_one('a[rel="next"]')
    if a and a.has_attr("href"):
        return urljoin(current_url, a["href"])
    a = soup.find("a", string=re.compile(r"(Selanjutnya|Berikutnya|Next|›|»)", re.I))
    if a and a.has_attr("href"):
        return urljoin(current_url, a["href"])
    # angka current+1
    cur = int(parse_qs(urlparse(current_url).query).get("page", ["1"])[0] or 1)
    for a in soup.find_all("a", href=True):
        t = (a.get_text(strip=True) or "")
        if t.isdigit() and int(t) == cur + 1:
            return urljoin(current_url, a["href"])
    return None

def collect_list_page(url: str) -> List[Dict[str, Any]]:
    """Ambil semua artikel dari 1 halaman list (Kompas/KompasTV)."""
    base = f"{urlparse(url).scheme}://{urlparse(url).netloc}"
    soup = get_soup(url)
    rows = []
    rows += extract_from_itemlist(soup, base)  # sering ada di Kompas.com; kadang ada di KompasTV
    rows += extract_from_dom(soup, base)       # fallback andalan (KompasTV)
    # parse tanggal awal + de-dup
    uniq = {}
    for r in rows:
        r["published"] = parse_id_date(r.get("published_raw"))
        uniq[r["url"]] = r
    return list(uniq.values()), soup

def crawl_section_all_pages(section_url: str,
                            need_date_from_detail: bool = True,
                            hard_cap: int = HARD_CAP) -> List[Dict[str, Any]]:
    """
    Jelajah semua halaman:
    - Ikuti 'next' via DOM; jika tidak ada, fallback ke ?page=N (2,3,...) sampai kosong.
    - Berhenti saat tidak ada halaman berikutnya atau mencapai hard_cap.
    """
    if RESPECT_ROBOTS and not is_allowed_by_robots(section_url):
        print("[robots] Disallowed:", section_url)
        return []

    results = []
    seen_urls = set()
    url = section_url
    page = 1

    while url and page <= hard_cap and url not in seen_urls:
        seen_urls.add(url)
        print(f"[crawl] page {page}: {url}")

        rows, soup = collect_list_page(url)
        if not rows and page > 1:
            print("[crawl] no rows → stop")
            break

        results.extend(rows)

        # 1) coba next dari DOM
        nxt = _find_next_by_dom(soup, url)

        # 2) kalau tidak ada, pakai fallback ?page=N
        if not nxt:
            nxt = _replace_page_param(url, page + 1)

        # guard: kalau next sama dgn current, berhenti
        if nxt == url:
            break

        url = nxt
        page += 1
        time.sleep(random.uniform(*DELAY_S))

    # De-dup global
    results = list({r["url"]: r for r in results}.values())

    # Lengkapi tanggal dari detail bila perlu
    if need_date_from_detail:
        for r in results:
            if r.get("published") is None:
                try:
                    d = parse_article_detail(r["url"])
                    r["title"] = r.get("title") or d.get("title")
                    if not r.get("published"):
                        r["published"] = parse_id_date(d.get("published_raw"))
                except Exception as e:
                    r["error_detail"] = str(e)
                time.sleep(random.uniform(*DELAY_S))

    return results

**_replace_page_param(url, page_num)**

Mengubah atau menambahkan parameter query page=N pada URL tanpa mengubah bagian path atau query lain.

**_extract_page_numbers_from_dom(soup)**

Mengumpulkan nomor halaman yang tersedia dari tautan pagination di halaman (misal angka halaman atau parameter page di URL).

**_find_next_by_dom(soup, current_url)**

Mencari tautan halaman berikutnya dengan memeriksa:

1. anchor dengan atribut rel="next",
2. anchor dengan teks seperti "Next", "Selanjutnya", "›", "»",
3. anchor dengan nomor halaman yang satu angka lebih besar dari halaman saat ini.

**collect_list_page(url)**

Mengambil semua artikel dari satu halaman daftar artikel dengan memanggil fungsi ekstraksi JSON-LD dan DOM, lalu mengurai tanggal publikasi dan menghilangkan duplikat.

**crawl_section_all_pages(section_url, need_date_from_detail=True, hard_cap=HARD_CAP)**

Fungsi utama untuk menjelajah (crawl) semua halaman daftar artikel pada sebuah section:

1. Mengikuti tautan halaman berikutnya yang ditemukan lewat DOM (rel=next atau teks next).
2. Jika tidak ada tautan next, menggunakan fallback dengan menambah parameter ?page=N.
3. Berhenti jika tidak ada halaman berikutnya, sudah mencapai batas maksimal halaman (hard_cap), atau URL halaman berikutnya sama dengan halaman saat ini.
4. Mengumpulkan semua artikel dari tiap halaman, menghilangkan duplikat, dan jika perlu melengkapi tanggal publikasi dengan mengambil detail artikel satu per satu.

---
**Eksekusi Crawler pada Satu Section**

Setelah semua fungsi pendukung dibuat, tahap berikutnya adalah menjalankan crawler utama.  
Di sini kita memanggil fungsi `crawl_section_all_pages` dengan parameter:  

- **SECTION_URL** → alamat awal section artikel yang ingin diambil.  
- **need_date_from_detail=True** → jika tanggal publikasi belum lengkap, crawler akan membuka halaman detail artikel.  
- **hard_cap=HARD_CAP** → batas maksimal jumlah halaman yang boleh dijelajahi, agar crawler tidak berjalan tanpa henti.  

Hasil eksekusi disimpan dalam variabel `rows`, yaitu daftar semua artikel yang berhasil dikumpulkan.  
Kemudian, kita cetak jumlah total item yang terkumpul beserta section URL-nya.  

In [ ]:
rows = crawl_section_all_pages(SECTION_URL, need_date_from_detail=True, hard_cap=HARD_CAP)
print(f"\nTotal items collected: {len(rows)}  (section: {SECTION_URL})")

[crawl] page 1: https://nasional.kompas.com/
[crawl] page 2: https://nasional.kompas.com/?page=2
[crawl] page 3: https://nasional.kompas.com/?page=3
[crawl] page 4: https://nasional.kompas.com/?page=4
[crawl] page 5: https://nasional.kompas.com/?page=5
[crawl] page 6: https://nasional.kompas.com/?page=6
[crawl] page 7: https://nasional.kompas.com/?page=7
[crawl] page 8: https://nasional.kompas.com/?page=8
[crawl] page 9: https://nasional.kompas.com/?page=9
[crawl] page 10: https://nasional.kompas.com/?page=10

Total items collected: 200  (section: https://nasional.kompas.com/)


---
**Penjelasan Fungsi - Data Duplicate Detection and Remover**

**1. canonicalize_url(u: str)**  

Membuat versi standar dari sebuah URL (kanonik), supaya variasi kecil dianggap sama.  
- Menghapus query string seperti ?utm atau ?source, dan juga fragment (#).  
- Menghilangkan suffix /amp di akhir path.  
- Merapikan tanda slash berlebih.  
- Memaksa URL menggunakan protokol https.  

**2. normalize_title(t: str)**  

Menormalkan judul artikel supaya mudah dibandingkan:  
- Mengubah huruf menjadi lowercase.  
- Menghapus tanda baca umum.  
- Memadatkan spasi berlebih.  

**3. dates_close(d1, d2, days=1)**  

Mengecek apakah dua tanggal berdekatan, dengan toleransi ±days (default = 1 hari).  
Jika salah satu tanggal kosong (None), otomatis dianggap tidak dekat.  

**4. deduplicate_rows(rows, fuzzy=True, fuzzy_threshold=0.92)**  

Fungsi utama untuk mendeteksi dan menghapus duplikat artikel.  
Aturan yang dipakai:  
1. Jika canonical URL sama → dianggap duplikat.  
2. Jika kombinasi judul normalisasi + tanggal sama → dianggap duplikat.  
3. Jika judul sangat mirip (fuzzy match ≥ threshold) dan tanggal berdekatan → dianggap duplikat.  

Menghasilkan:  
- unique: daftar artikel unik.  
- dup_report: laporan artikel duplikat (mana yang disimpan, mana yang dibuang, beserta alasannya).  

In [ ]:
#@title Data Duplicate Detection and Remover

def canonicalize_url(u: str) -> str:
    """
    Bikin URL kanonik supaya varian yang sama (utm, source, amp) dianggap satu.
    - drop query & fragment
    - buang suffix /amp
    - rapikan slash
    - paksa https
    """
    if not u:
        return ""
    p = urlparse(u)
    path = re.sub(r"/amp/?$", "/", p.path or "/")
    path = re.sub(r"//+", "/", path)
    if path != "/" and path.endswith("/"):
        path = path[:-1]
    canon = urlunparse(("https", p.netloc.lower(), path, "", "", ""))  # tanpa query/fragment
    return canon

def normalize_title(t: str) -> str:
    """Lowercase, hilangkan tanda baca umum, padatkan spasi."""
    if not t:
        return ""
    t = t.lower()
    t = re.sub(r"[^\w\s]", " ", t, flags=re.UNICODE)
    t = re.sub(r"\s+", " ", t).strip()
    return t

def dates_close(d1, d2, days=1) -> bool:
    """True jika dua tanggal sama/berdekatan (±days). None dianggap tidak dekat."""
    if d1 is None or d2 is None:
        return False
    return abs((d1 - d2).days) <= days

def deduplicate_rows(rows, fuzzy=True, fuzzy_threshold=0.92):
    """
    Kembalikan (unique_rows, dup_report)
    - unique_rows: list artikel unik
    - dup_report : list {keep, drop, reason}
    Aturan:
      1) Sama canonical URL → duplikat
      2) Sama (title_norm, published) → duplikat
      3) Fuzzy: mirip judul (>=threshold) & tanggal dekat → duplikat
    """
    unique = []
    dup_report = []

    url_index = {}                 # canon_url -> idx unique
    title_date_index = {}          # (title_norm, published) -> idx unique

    for r in rows:
        item = dict(r)  # salin agar aman
        cu = canonicalize_url(item.get("url", ""))
        tn = normalize_title(item.get("title", ""))
        dt = item.get("published")  # diasumsikan sudah tipe date (dari parse_id_date)

        # Rule 1: exact canonical URL
        if cu and cu in url_index:
            dup_report.append({"reason": "same_canonical_url", "keep": unique[url_index[cu]], "drop": item})
            continue

        # Rule 2: same (title_norm, date)
        key_td = (tn, dt) if tn and dt else None
        if key_td and key_td in title_date_index:
            dup_report.append({"reason": "same_title_and_date", "keep": unique[title_date_index[key_td]], "drop": item})
            continue

        # Rule 3: fuzzy title (same/close date)
        matched = False
        if fuzzy and tn:
            for idx, kept in enumerate(unique):
                if not dates_close(dt, kept.get("published")):
                    continue
                kept_tn = normalize_title(kept.get("title", ""))
                if not kept_tn:
                    continue
                ratio = difflib.SequenceMatcher(None, tn, kept_tn).ratio()
                if ratio >= fuzzy_threshold:
                    dup_report.append({
                        "reason": f"fuzzy_title_sim>={fuzzy_threshold} (score={ratio:.2f})",
                        "keep": kept, "drop": item
                    })
                    matched = True
                    break
        if matched:
            continue

        # Keep this item
        unique.append(item)
        if cu:
            url_index[cu] = len(unique) - 1
        if key_td:
            title_date_index[key_td] = len(unique) - 1

    return unique, dup_report

**Proses Deduplikasi Data**

Setelah semua artikel dari section berhasil dikumpulkan, langkah selanjutnya adalah menghapus data ganda (duplicate).  
Untuk itu kita gunakan fungsi `deduplicate_rows` dengan parameter:  

- **rows** → data mentah hasil crawling.  
- **fuzzy=True** → aktifkan pencocokan judul mirip (fuzzy matching).  
- **fuzzy_threshold=0.92** → ambang kemiripan judul, misalnya ≥92% dianggap sama.  

Fungsi ini menghasilkan dua keluaran:  
- **unique_rows** → daftar artikel yang benar-benar unik.  
- **dup_report** → laporan artikel yang dianggap duplikat beserta alasannya.  

Akhirnya, kita cetak ringkasan: berapa jumlah data sebelum deduplikasi, berapa yang unik, dan berapa yang terdeteksi duplikat.  

In [ ]:
unique_rows, dup_report = deduplicate_rows(rows, fuzzy=True, fuzzy_threshold=0.92)
print(f"rows before: {len(rows)} | unique: {len(unique_rows)} | removed dup: {len(dup_report)}")

rows before: 200 | unique: 199 | removed dup: 1


---
**Filtering Function**

Setelah data terkumpul dan deduplikasi, biasanya kita butuh **menyaring artikel** sesuai kriteria tertentu.  
Fungsi filtering ini punya beberapa komponen:

1. **within_range(d, start, end)**  
   Mengecek apakah tanggal artikel berada dalam rentang tertentu.  
   - Jika `d` (tanggal) kosong, keputusan tergantung pada konfigurasi `INCLUDE_WITHOUT_DATE`.  
   - Jika ada batas awal (`start`) atau akhir (`end`), artikel dicek apakah sesuai dengan rentang tanggal tersebut.  

2. **_join_text_for_match(item, mode)**  
   Membuat string gabungan sesuai mode pencarian.  
   - `"title"` → hanya judul.  
   - `"title+summary"` → judul + ringkasan.  
   - `"title+text"` → judul + ringkasan + isi teks.  
   Hasil teks bisa dibuat lowercase jika pencarian tidak case-sensitive.  

3. **_matches_keywords(text, keywords)**  
   Mengecek apakah ada kata kunci yang muncul dalam teks.  
   - Jika daftar keyword kosong, otomatis dianggap cocok.  
   - Kalau ada keyword, teks diperiksa satu per satu.  

In [ ]:
#@title Filtering Function

def within_range(d, start, end):
    if d is None:
        return bool(INCLUDE_WITHOUT_DATE and not start and not end)
    s = datetime.strptime(start, "%Y-%m-%d").date() if start else None
    e = datetime.strptime(end, "%Y-%m-%d").date() if end else None
    if s and d < s: return False
    if e and d > e: return False
    return True

def _join_text_for_match(item, mode):
    parts = []
    if mode in ("title", "title+summary", "title+text"):
        parts.append(item.get("title","") or "")
    if mode in ("title+summary", "title+text"):
        parts.append(item.get("summary","") or "")
    if mode == "title+text":
        parts.append(item.get("text","") or "")
    txt = " ".join(p for p in parts if p)
    return txt if CASE_SENSITIVE else txt.lower()

def _matches_keywords(text, keywords):
    if not keywords:
        return True
    hay = text if CASE_SENSITIVE else text.lower()
    for kw in keywords:
        needle = kw if CASE_SENSITIVE else kw.lower()
        if needle in hay:
            return True
    return False

---
**Penyaringan Artikel dengan Keyword dan Rentang Tanggal**

Setelah data artikel terkumpul, kita lakukan tahap akhir berupa filtering agar hanya artikel yang relevan yang tersisa.  

Langkah filtering meliputi:  
1. **Pemeriksaan keyword** → menggunakan `_matches_keywords` untuk cek apakah artikel mengandung salah satu kata kunci yang ditentukan.  
2. **Pemeriksaan tanggal tersedia** → menghitung berapa artikel yang memiliki tanggal publikasi (tidak None).  
3. **Pemeriksaan rentang tanggal** → menggunakan `within_range` untuk cek apakah artikel terbit dalam rentang `START_DATE` hingga `END_DATE`.  

Hanya artikel yang lolos **dua syarat utama** (mengandung keyword dan berada dalam rentang tanggal) yang dimasukkan ke dalam list `filtered`.  

Akhirnya, dicetak ringkasan jumlah artikel:  
- total artikel hasil crawling,  
- berapa yang cocok dengan keyword,  
- berapa yang memiliki tanggal,  
- berapa yang berada dalam rentang,  
- dan total artikel yang tersaring setelah filter.  

In [ ]:
matched_kw = have_date = in_range = 0
filtered = []

for r in rows:
    txt = _join_text_for_match(r, MATCH_IN)
    ok_kw   = _matches_keywords(txt, KEYWORDS);               matched_kw += bool(ok_kw)
    ok_date = r.get("published") is not None;                 have_date  += bool(ok_date)
    ok_rng  = within_range(r.get("published"), START_DATE, END_DATE); in_range += bool(ok_rng)
    if ok_kw and ok_rng:
        filtered.append(r)

print(f"After crawl: {len(rows)} items | matched_kw: {matched_kw} | have_date: {have_date} | in_range: {in_range} | After filter: {len(filtered)}")

After crawl: 200 items | matched_kw: 5 | have_date: 200 | in_range: 137 | After filter: 2


Dari output tersebut terlihat bahwa kita mendapat 200 berita dengan 4 berita yang sesuai dengan keyword, 200 berita yang tanggal publis nya tersedia, 143 berita yang berada pada tanggal 1 September - 4 September sesuai dengan pengaturan filternya, dan ada 2 berita yang keyword dan tanggalnya sesuai dengan filter.

---
**Menampilkan Hasil Artikel yang Sudah Difilter**

Tahap terakhir adalah melihat hasil artikel yang sudah lolos filter.  
Caranya:  

1. **Sortir artikel** → daftar `filtered` diurutkan berdasarkan tanggal publikasi (`published`), dari terbaru ke terlama.  
   - Jika ada artikel tanpa tanggal, digunakan nilai minimal `date.min` supaya tetap bisa diurutkan.  

2. **Cetak artikel teratas** → ditampilkan maksimal 20 artikel pertama dari hasil sortir.  
   Untuk tiap artikel ditampilkan informasi:  
   - tanggal publikasi,  
   - judul,  
   - URL,  
   - ringkasan (summary) jika tersedia, dipotong maksimal 180 karakter agar rapi.  

In [ ]:
filtered_sorted = sorted(filtered, key=lambda x: x.get("published") or date.min, reverse=True)
print("Scrapping Results:")
for it in filtered_sorted[:20]:
    print("-"*130)
    print("published :", it.get("published"))
    print("title     :", it.get("title"))
    print("url       :", it.get("url"))
    if it.get("summary"):
        s = it["summary"]; print("summary   :", (s[:180] + "...") if len(s) > 180 else s)

Scrapping Results:
----------------------------------------------------------------------------------------------------------------------------------
published : 2025-04-09
title     : Profil Rusdi Masse, Wakil Ketua Komisi III Pengganti Sahroni, Pernah Jadi Bupati Termuda Nasional 4 September 2025
url       : https://nasional.kompas.com/read/2025/09/04/11133361/profil-rusdi-masse-wakil-ketua-komisi-iii-pengganti-sahroni-pernah-jadi
----------------------------------------------------------------------------------------------------------------------------------
published : 2025-04-09
title     : Gantikan Ahmad Sahroni, Rusdi Masse Mappassesu Dilantik Jadi Wakil Ketua Komisi III Nasional 4 September 2025
url       : https://nasional.kompas.com/read/2025/09/04/10055731/gantikan-ahmad-sahroni-rusdi-masse-mappassesu-dilantik-jadi-wakil-ketua


---
Selanjutnya kita bisa menyimpannya ke dalam csv dan siap untuk diolah lebih lanjut.

In [ ]:
rows_csv = [{
    "published": (it.get("published").isoformat()
                  if hasattr(it.get("published"), "isoformat") else it.get("published")),
    "title": it.get("title"),
    "url": it.get("url")
} for it in filtered_sorted]

df = pd.DataFrame(rows_csv)

# df.to_csv("/content/kompas_filtered.csv", index=False, encoding="utf-8")
# print("CSV disimpan ke /content/kompas_filtered.csv, rows:", len(df))

In [ ]:
df.head()

,published,title,url
0,2025-04-09,"Profil Rusdi Masse, Wakil Ketua Komisi III Pen...",https://nasional.kompas.com/read/2025/09/04/11...
1,2025-04-09,"Gantikan Ahmad Sahroni, Rusdi Masse Mappassesu...",https://nasional.kompas.com/read/2025/09/04/10...


---
**Note:**
Dikarenakan pada laman artikel https://nasional.kompas.com/ maksimal hanya menampilkan 10 page, jadi jika ada berita dengan tanggal lebih lama dibanding berita terakhir di halaman ke-10 maka berita tersebut tidak dapat ditampilkan.

Sekian, arigatou..

<p align="align-left">
  <img src="https://media1.tenor.com/m/u-0BLG9HxAQAAAAC/mambo-matikanetannhauser.gif" width="320" alt="Mambo">
</p>